In [ ]:
import numpy as np
import tensorflow
import matplotlib.pyplot as plt

In [ ]:
train_dir = '/content/drive/My Drive/Colab Notebooks/dataset/train'
validation_dir = '/content/drive/My Drive/Colab Notebooks/dataset/validation'
test_dir = '/content/drive/My Drive/Colab Notebooks/dataset/test'
image_size = 224

In [ ]:
from tensorflow.keras.applications import VGG16
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
for layer in vgg_conv.layers[:]:
    layer.trainable = False
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
model = Sequential()
model.add(vgg_conv)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
batchSize = 32
train_generator = datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batchSize, class_mode='categorical', shuffle=True)
validation_generator = datagen.flow_from_directory(validation_dir, target_size=(image_size, image_size), batch_size=batchSize, class_mode='categorical', shuffle=False)
test_generator = datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batchSize, class_mode='categorical', shuffle=False)

In [ ]:
id = 0
print('The shape of train images: {}'.format(train_generator[id][0][0].shape))
plt.imshow(train_generator[id][0][0])
plt.axis('off')
train_image_label_id = np.argmax(train_generator[id][1][0])
classes_list = list(train_generator.class_indices.keys())
plt.title('Class name: {}'.format(classes_list[train_image_label_id]))
plt.show()

In [ ]:
id = 0
print('The shape of validation images: {}'.format(validation_generator[id][0][0].shape))
plt.imshow(validation_generator[id][0][0])
plt.axis('off')
val_image_label_id = np.argmax(validation_generator[id][1][0])
classes_list = list(validation_generator.class_indices.keys())
plt.title('Class name: {}'.format(classes_list[val_image_label_id]))
plt.show()

In [ ]:
id = 0
print('The shape of test images: {}'.format(test_generator[id][0][0].shape))
plt.imshow(test_generator[id][0][0])
plt.axis('off')
tes_image_label_id = np.argmax(test_generator[id][1][0])
classes_list = list(test_generator.class_indices.keys())
plt.title('Class name: {}'.format(classes_list[tes_image_label_id]))
plt.show()

In [ ]:
from tensorflow.keras import optimizers
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=['accuracy'])
history = model.fit(train_generator, steps_per_epoch=train_generator.samples//train_generator.batch_size, epochs=20, validation_data=validation_generator, validation_steps=validation_generator.samples//validation_generator.batch_size, verbose=1)

In [ ]:
plt.figure(figsize=[8, 6])
plt.plot(history.history['loss'], 'r', linewidth=3.0)
plt.plot(history.history['val_loss'], 'b', linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'], fontsize=18)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.title('Loss Curves', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=[8, 6])
plt.plot(history.history['accuracy'], 'r', linewidth=3.0)
plt.plot(history.history['val_accuracy'], 'b', linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.title('Accuracy Curves', fontsize=16)
plt.show()

In [ ]:
model_json = model.to_json()
with open('/content/drive/My Drive/Colab Notebooks/tl_model_comb.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('/content/drive/My Drive/Colab Notebooks/tl_model_comb.h5')

In [ ]:
from tensorflow.keras.models import model_from_json
json_file = open('/content/drive/My Drive/Colab Notebooks/tl_model_comb.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/content/drive/My Drive/Colab Notebooks/tl_model_comb.h5')

In [ ]:
fnames = test_generator.filenames
ground_truth = test_generator.classes
label2index = test_generator.class_indices
idx2label = dict((v,k) for k,v in label2index.items())
predictions = loaded_model.predict(test_generator, steps=(test_generator.samples//test_generator.batch_size)+1, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
errors = np.where(predicted_classes != ground_truth)[0]
print('No of errors = {}/{}'.format(len(errors), test_generator.samples))

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
    print('Original label: {}, Prediction: {}, Confidence: {:.3f}'.format(fnames[errors[i]].split('/')[0], pred_label, predictions[errors[i]][pred_class]))
    original = load_img('{}/{}'.format(test_dir, fnames[errors[i]]))
    plt.imshow(original)
    plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
filename = '/content/drive/My Drive/Colab Notebooks/pumpkin.jpg'
original = load_img(filename, target_size=(224, 224))
numpy_image = img_to_array(original)
image_batch = np.expand_dims(numpy_image, axis=0)
plt.imshow(np.uint8(image_batch[0]))
plt.show()
prediction = np.argmax(loaded_model.predict(image_batch), axis=1)
probability = loaded_model.predict(image_batch)
predicted_class = np.argmax(probability[0])
predict_label = idx2label[predicted_class]
print('Prediction: {}'.format(predict_label))